# Look at the pH of the system

In [ ]:
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!rsync -aP pi@192.168.0.15:growControl_Data/* ~/growControl_Data

In [ ]:
desktop_root_path = "/home/neil/"
pi_root_path = "/home/pi/"
if os.path.isdir(desktop_root_path):
    data_path = os.path.join(desktop_root_path,"growControl_Data")
    analysis_path = os.path.join(desktop_root_path,"growControl","Analysis")
elif os.path.isdir(pi_root_path):
    data_path = os.path.join(pi_path,"growControl_Data")
    analysis_path = os.path.join(pi_path,"growControl","Analysis")
else:
    raise FileNotFoundError("No valid path to the data was found!")

In [ ]:
data_files = {}
for d in os.listdir(data_path):
    if not os.path.isfile(os.path.join(data_path,d)):
        continue
    root = d[:d.rfind("_")]
    full_path = os.path.join(data_path,d)
    if root not in data_files:
        data_files[root] = [full_path]
    else:
        data_files[root].append(full_path)

for key in data_files:
    print(key)
    print("\t{}".format(data_files[key][0]))
    print("\t{}".format(data_files[key][-1]))
    #for file in data_files[key]:
    #    print("\t{}".format(file))

In [ ]:
def load_df(root_name):
    li = []
    for fname in sorted(data_files[root_name])[-20:]:
        df_temp = pd.read_csv(fname,index_col=None,header=0,skiprows=lambda x: (x!=0) and not x%10)
        li.append(df_temp)
    df = pd.concat(li,axis=0,ignore_index=True)
    
    for column in df.columns:
        df = df[df[column] != "None"]
        if column != "datetime_timezone":
            df[column] = df[column].astype(float)
        
    df["datetime_timezone"] = pd.to_datetime(df["datetime_timezone"])
    df.sort_values("datetime_timezone",inplace=True)
    df.reset_index(inplace=True,drop=True)
    return df

In [ ]:
df_ph = load_df("sensor_ph_bin1")
df_control_ph = load_df("controller_ph_pump")

#plt.rcParams["figure.figsize"] = (50,25)
plt.rcParams["figure.figsize"] = (20,10)
fig, ax1 = plt.subplots()
ax1.set_xlabel("Date")
ax1.set_ylabel("pH")

ax1.plot(df_ph["datetime_timezone"],df_ph["ph_raw"],label="raw")
ax1.plot(df_ph["datetime_timezone"],df_ph["ph_avg"],label="ph_avg")

#ax1.plot(df_ph["datetime"],df_ph["ph_avg_ewm_.1"],label="average_factor=0.9 => Current Value")
#ax1.plot(df_ph["datetime"],df_ph["ph_avg_ewm_.01"],label="average_factor=0.99 => Changing To")
#ax1.plot(df_ph["datetime"],df_ph["ph_avg_ewm_.001"],label="average_factor=0.999")

datetime_minmax = [df_ph["datetime_timezone"].min(),df_ph["datetime_timezone"].max()]
ax1.plot(datetime_minmax,[5.8,5.8],label="pH Min")
ax1.plot(datetime_minmax,[6.2,6.2],label="pH Max")

ax1.legend(loc="upper left")
ax1.minorticks_on()
ax1.grid(True,which="major",linewidth=1)
ax1.grid(True,which="minor",linewidth=.5)
plt.ylim((5.5,8.))
plt.xlim(datetime_minmax)

ax2 = ax1.twinx()
ax2.set_ylabel("pH Adjustment Volume\n[ml]")
ax2.plot(df_control_ph["datetime_timezone"],df_control_ph["ph_down_volume"],'r',alpha=.2,label="pH Down")
ax2.plot(df_control_ph["datetime_timezone"],df_control_ph["ph_up_volume"],'b',alpha=.2,label="pH Up")
ax2.legend(loc="upper right")
plt.ylim((0,2.0))
plt.xlim(datetime_minmax)
plt.savefig(os.path.join(analysis_path,"ph_data_{}.png".format(datetime.datetime.now().strftime("%Y%m%dT%H%M%S"))))
plt.show()

In [ ]:
def plotData(dataFrames,listOfColumnsInDataFrames):
    
    
    plt.rcParams["figure.figsize"] = (20,10)
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Date")
    
    y_lim = None
    
    for df,columns in zip(dataFrames,listOfColumnsInDataFrames):
        ax1.set_ylabel(columns)
        for column in columns:
            ax1.plot(df["datetime_timezone"],df[column],label=column)
            if y_lim is None:
                y_lim = [df[column].min(),df[column].max()]
            else:
                y_lim[0] = min(y_lim[0],df[column].min())
                y_lim[1] = min(y_lim[1],df[column].max())
    
    datetime_minmax = [df["datetime_timezone"].min(),df["datetime_timezone"].max()]
    plt.xlim(datetime_minmax)
    
    delta = y_lim[1]-y_lim[0]
    y_lim = [y_lim[0]-.1*delta,y_lim[1]+.1*delta]
    plt.ylim(y_lim)
    
    ax1.minorticks_on()
    ax1.grid(True,which="major",linewidth=1)
    ax1.grid(True,which="minor",linewidth=.5)
    
    
    ax1.legend(loc="upper right")
    plt.show()

In [ ]:
humidity_temp_ambient = load_df("humidity_temp_ambient")
humidity_temp_grow = load_df("humidity_temp_grow")

plotData([humidity_temp_ambient,humidity_temp_grow],[["temperature_raw"],["temperature_raw"]])

In [ ]:
humidity_temp_ambient = load_df("humidity_temp_ambient")
humidity_temp_grow = load_df("humidity_temp_grow")

plotData([humidity_temp_ambient,humidity_temp_grow],[["relative_humidity_raw"],["relative_humidity_raw"]])

In [ ]:
volume = load_df("sensor_volume")

plotData([volume],[["volume_raw","volume_avg"]])